In [293]:
#Elasticsearch connector
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import eland as ed
#Data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import eland as ed
import numpy as np
import matplotlib.pyplot as plt
import math
# Disable warnings"
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [294]:
es = Elasticsearch(['https://192.168.1.181:9200'],
                   
ca_certs=False,verify_certs=False, http_auth=('jupyter','password'))

# defining the search statement to get all records in an index
searchContext = Search(using=es, index='*:so-*', doc_type='doc')

es.info()

ObjectApiResponse({'name': 'sohunter', 'cluster_name': 'sohunter', 'cluster_uuid': '586orLzYSvSqAfvBdjLTfQ', 'version': {'number': '7.16.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b937c44140b6559905130a8650c64dbd0879cfb', 'build_date': '2021-12-18T19:42:46.604893745Z', 'build_snapshot': False, 'lucene_version': '8.10.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [295]:
query_since_ts = 0
#body_filter = '{"query":{"bool":{"filter":[{"range":{@timestamp":{"gt":' + str(query_since_ts) + '}}}]}},"sort":[{"@timstamp":{"order":"dsc"}}]}'
body_filter = '{"query":{"match_all":{}},"sort":[{"ts":{"order":"desc"}}]}'

In [296]:
#s = searchContext.query('query_string', query='message:("ts" AND "id.orig_h" AND "id.resp_h" AND "host" AND "id.resp_p" AND "method")')
 
#s = searchContext.query('query_string', query=('destination.port("*")'))


#s = searchContext.query('query_string', query='(destination.port:"*" OR http.virtual_host:"*")')

#s = searchContext.query('query_string', query='http.virtual_host:*')

s = searchContext.query('query_string', query='http.method:*')

In [297]:
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 600)
pd.set_option("display.max_columns", None)


#base response query
#response = s.execute()
#if response.success():
#  df = pd.DataFrame((d.to_dict() for d in s.scan()))
#json_struct = json.loads(df.to_json(orient="records"))
#df_flat = pd.io.json.json_normalize(json_struct)
#df_flat.head(20)

#response = s.execute()
#if response.success():
    #df = pd.DataFrame(([d['@timestamp'], d['source']['ip'], d['destination']['ip'], d['http']['virtual_host'], d['destination']['port'], d['http']['method']] for d in s))
#df.columns=['Timestamp','Source.ip','Destination.ip','Host','Destination.port','Method']    
#df.astype({'Destination.Port': 'int64'})
#display(df)
#df.info()

In [298]:
response = s.execute()
if response.success():
    df = pd.DataFrame(([d['@timestamp'], d['source']['ip'], d['destination']['ip'], d['http']['virtual_host'], d['destination']['port'], d['http']['method']] for d in s))
df.columns=['Timestamp','Source.ip','Destination.ip','Host','Destination.port','Method']    
#df.astype({'Destination.Port': 'int64'})
display(df)

df.info()

,Timestamp,Source.ip,Destination.ip,Host,Destination.port,Method
0,2021-11-14T22:05:06.018Z,192.168.1.154,209.197.3.8,msedge.b.tlu.dl.delivery.mp.microsoft.com,80,GET
1,2021-11-14T22:05:05.010Z,192.168.1.154,209.197.3.8,msedge.b.tlu.dl.delivery.mp.microsoft.com,80,GET
2,2021-11-14T22:04:58.886Z,192.168.1.154,209.197.3.8,msedge.b.tlu.dl.delivery.mp.microsoft.com,80,GET
3,2021-11-14T21:55:43.466Z,192.168.1.154,156.112.111.142,ocsp.disa.mil,80,GET
4,2021-11-14T21:55:41.797Z,192.168.1.154,23.40.205.26,officecdn.microsoft.com.edgesuite.net,80,GET
5,2021-11-14T21:55:41.557Z,192.168.1.154,23.219.216.116,officecdn.microsoft.com,80,GET
6,2021-11-14T21:55:40.883Z,192.168.1.154,23.219.216.116,officecdn.microsoft.com,80,GET
7,2021-11-14T21:55:40.520Z,192.168.1.154,23.219.216.116,officecdn.microsoft.com,80,HEAD
8,2021-11-14T21:55:39.895Z,192.168.1.154,23.40.205.26,officecdn.microsoft.com.edgesuite.net,80,HEAD
9,2021-11-14T21:55:39.097Z,192.168.1.154,23.40.205.26,officecdn.microsoft.com.edgesuite.net,80,HEAD


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Timestamp         10 non-null     object
 1   Source.ip         10 non-null     object
 2   Destination.ip    10 non-null     object
 3   Host              10 non-null     object
 4   Destination.port  10 non-null     int64 
 5   Method            10 non-null     object
dtypes: int64(1), object(5)
memory usage: 608.0+ bytes


In [299]:
# assign field/column names to variables
f_timestamp = 'Timestamp'
f_src_ip = 'Source.ip'
f_dst_ip = 'Destination.ip'
f_dst_host = 'Host'
f_dst_port = 'Destination.port'
f_http_method = 'Method'
f_delimiter = '\t'

columns_to_filter = [f_timestamp, f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_http_method]
columns_to_groupby = [f_src_ip, f_dst_ip, f_dst_host, f_dst_port, f_http_method]
# columns to display after the analysis
columns_to_display = ['tsScore','conn_count',f_src_ip,f_dst_ip,f_dst_host,f_http_method, f_dst_port,'deltas']

In [300]:
# If you have a large dateset, using groupby and aggregate(list) might be slow. Consider using dask
df = df.groupby(columns_to_groupby).agg(list)
df.head(10)

Timestamp
Source.ip     Destination.ip  Host                                      Destination.port Method                                                                                
192.168.1.154 156.112.111.142 ocsp.disa.mil                             80               GET                                                         [2021-11-14T21:55:43.466Z]
              209.197.3.8     msedge.b.tlu.dl.delivery.mp.microsoft.com 80               GET     [2021-11-14T22:05:06.018Z, 2021-11-14T22:05:05.010Z, 2021-11-14T22:04:58.886Z]
              23.219.216.116  officecdn.microsoft.com                   80               GET                               [2021-11-14T21:55:41.557Z, 2021-11-14T21:55:40.883Z]
                                                                                         HEAD                                                        [2021-11-14T21:55:40.520Z]
              23.40.205.26    officecdn.microsoft.com.edgesuite.net     80               GET                                                         [2021-11-14T21:55:41.797Z]
                                                                                         HEAD                              [2021-11-14T21:55:39.895Z, 2021-11-14T21:55:39.097Z]

In [302]:
df.reset_index(inplace=True)
df.head(5)

,index,Source.ip,Destination.ip,Host,Destination.port,Method,Timestamp
0,0,192.168.1.154,156.112.111.142,ocsp.disa.mil,80,GET,[2021-11-14T21:55:43.466Z]
1,1,192.168.1.154,209.197.3.8,msedge.b.tlu.dl.delivery.mp.microsoft.com,80,GET,"[2021-11-14T22:05:06.018Z, 2021-11-14T22:05:05.010Z, 2021-11-14T22:04:58.886Z]"
2,2,192.168.1.154,23.219.216.116,officecdn.microsoft.com,80,GET,"[2021-11-14T21:55:41.557Z, 2021-11-14T21:55:40.883Z]"
3,3,192.168.1.154,23.219.216.116,officecdn.microsoft.com,80,HEAD,[2021-11-14T21:55:40.520Z]
4,4,192.168.1.154,23.40.205.26,officecdn.microsoft.com.edgesuite.net,80,GET,[2021-11-14T21:55:41.797Z]


In [303]:
# create a new column 'conn_count', and for each row in the 'ts' column, apply a function and assign the returned value to the 'conn_count' column
df['conn_count'] = df[f_timestamp].apply(lambda x: len(x))
df.head(2)

,index,Source.ip,Destination.ip,Host,Destination.port,Method,Timestamp,conn_count
0,0,192.168.1.154,156.112.111.142,ocsp.disa.mil,80,GET,[2021-11-14T21:55:43.466Z],1
1,1,192.168.1.154,209.197.3.8,msedge.b.tlu.dl.delivery.mp.microsoft.com,80,GET,"[2021-11-14T22:05:06.018Z, 2021-11-14T22:05:05.010Z, 2021-11-14T22:04:58.886Z]",3


In [304]:
#Filter out traffic where the connection count is quite small
df = df.loc[df['conn_count'] > 0]
df.shape

(6, 8)

In [305]:
df[f_timestamp] = df[f_timestamp].apply(lambda x: sorted(x))
df.head()

,index,Source.ip,Destination.ip,Host,Destination.port,Method,Timestamp,conn_count
0,0,192.168.1.154,156.112.111.142,ocsp.disa.mil,80,GET,[2021-11-14T21:55:43.466Z],1
1,1,192.168.1.154,209.197.3.8,msedge.b.tlu.dl.delivery.mp.microsoft.com,80,GET,"[2021-11-14T22:04:58.886Z, 2021-11-14T22:05:05.010Z, 2021-11-14T22:05:06.018Z]",3
2,2,192.168.1.154,23.219.216.116,officecdn.microsoft.com,80,GET,"[2021-11-14T21:55:40.883Z, 2021-11-14T21:55:41.557Z]",2
3,3,192.168.1.154,23.219.216.116,officecdn.microsoft.com,80,HEAD,[2021-11-14T21:55:40.520Z],1
4,4,192.168.1.154,23.40.205.26,officecdn.microsoft.com.edgesuite.net,80,GET,[2021-11-14T21:55:41.797Z],1


In [306]:
# Convert list into a Series object, get time delta, convert the result back into a list and assign it to the 'deltas' column
df['deltas'] = df[f_timestamp].apply(lambda x: pd.Series(x).diff().dt.seconds.dropna().tolist())
df.head(2)

AttributeError: Can only use .dt accessor with datetimelike values

In [292]:
df['tsLow'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 20))
df['tsMid'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 50))
df['tsHigh'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 80))
df['tsBowleyNum'] = df['tsLow'] + df['tsHigh'] - 2*df['tsMid']
df['tsBowleyDen'] = df['tsHigh'] - df['tsLow']
df['tsSkew'] = df[['tsLow','tsMid','tsHigh','tsBowleyNum','tsBowleyDen']].apply(
    lambda x: x['tsBowleyNum'] / x['tsBowleyDen'] if x['tsBowleyDen'] != 0 and x['tsMid'] != x['tsLow'] and x['tsMid'] != x['tsHigh'] else 0.0, axis=1
    )
df['tsMadm'] = df['deltas'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))
df['tsConnDiv'] = df[f_timestamp].apply(lambda x: (x[-1].to_pydatetime() - x[0].to_pydatetime()).seconds / 10)

KeyError: 'deltas'